In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input #type:ignore
import matplotlib.pyplot as plt
import os

# CONFIG
img_height, img_width = 180, 180
batch_size = 32
num_classes = 26  # 00 to 25

# --- LOAD TEMPLATE IMAGES ---
template_images = {}
for class_idx in range(num_classes):
    path = f"./templates/{class_idx:02d}.png"
    img = tf.keras.utils.load_img(path, target_size=(img_height, img_width))
    img = tf.keras.utils.img_to_array(img) / 255.0
    template_images[class_idx] = img

# --- DATASET LOADING ---
def load_dataset(path):
    return tf.keras.utils.image_dataset_from_directory(
        path,
        shuffle=True,
        image_size=(img_height, img_width),
        batch_size=batch_size,
        label_mode='int'
    )

data_train = load_dataset('./dataset/newDatasetSplit/train')
data_val = load_dataset('./dataset/newDatasetSplit/val')

# --- MAP EACH INPUT TO ITS TEMPLATE IMAGE ---
def map_to_target(img, label):
    target_img = tf.convert_to_tensor([template_images[int(label[i].numpy())] for i in range(len(label))], dtype=tf.float32)
    return img, target_img

def tf_wrapper(img, label):
    img, target_img = tf.py_function(map_to_target, inp=[img, label], Tout=(tf.float32, tf.float32))
    img.set_shape((None, img_height, img_width, 3))
    target_img.set_shape((None, img_height, img_width, 3))
    return img, target_img

paired_train = data_train.map(tf_wrapper)
paired_val = data_val.map(tf_wrapper)

# --- MODEL: ENCODER-DECODER ---
def build_image_to_image_model():
    inputs = Input(shape=(img_height, img_width, 3))
    x = layers.Rescaling(1./255)(inputs)

    # Encoder
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
    skip1 = x
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    skip2 = x
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    skip3 = x
    x = layers.MaxPooling2D()(x)

    # Decoder
    x = layers.Conv2DTranspose(128, 3, strides=2, activation='relu', padding='same')(x)
    x = layers.Cropping2D(((0, 0), (0, 0)))(x)  # Adjust shape to match skip3
    x = layers.Concatenate()([x, skip3])
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)

    x = layers.Conv2DTranspose(64, 3, strides=2, activation='relu', padding='same')(x)
    x = layers.Cropping2D(((0, 0), (0, 0)))(x)  # Adjust shape to match skip2
    x = layers.Concatenate()([x, skip2])
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)

    x = layers.Conv2DTranspose(32, 3, strides=2, activation='relu', padding='same')(x)
    x = layers.Cropping2D(((0, 0), (0, 0)))(x)  # Adjust shape to match skip1
    x = layers.Concatenate()([x, skip1])
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)

    outputs = layers.Conv2D(3, 3, activation='sigmoid', padding='same')(x)

    return models.Model(inputs, outputs)

# --- COMPILE ---
model = build_image_to_image_model()
model.compile(optimizer='adam', loss='mse')

# --- TRAIN ---
EPOCHS = 32
history = model.fit(
    paired_train,
    validation_data=paired_val,
    epochs=EPOCHS
)

# --- PLOT RESULTS ---
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()

plt.subplot(1, 2, 2)
for image, label in data_val.take(1):
    pred = model.predict(image)
    for i in range(4):
        plt.subplot(2, 4, i + 1)
        plt.imshow(image[i].numpy().astype("uint8"))
        plt.axis('off')
        plt.title("Input")
        plt.subplot(2, 4, i + 5)
        plt.imshow(pred[i])
        plt.axis('off')
        plt.title("Output")

plt.tight_layout()
plt.show()

# --- SAVE MODEL ---
model.save("image_to_image_model.keras")
print("Model saved!")

Found 17125 files belonging to 25 classes.
Found 3675 files belonging to 25 classes.


ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 44, 44, 128), (None, 45, 45, 128)]

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input  # type:ignore
import matplotlib.pyplot as plt
import os

# --- Enable GPU Memory Growth ---
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# CONFIG
img_height, img_width = 128, 128
batch_size = 8
num_classes = 26  # 00 to 25

# --- LOAD TEMPLATE IMAGES ---
template_images = {}
for class_idx in range(num_classes):
    path = f"./templates/{class_idx:02d}.png"
    img = tf.keras.utils.load_img(path, target_size=(img_height, img_width))
    img = tf.keras.utils.img_to_array(img) / 255.0
    template_images[class_idx] = img

# --- DATASET LOADING ---
def load_dataset(path):
    return tf.keras.utils.image_dataset_from_directory(
        path,
        shuffle=True,
        image_size=(img_height, img_width),
        batch_size=batch_size,
        label_mode='int'
    )

data_train = load_dataset('./dataset/newDatasetSplit/train')
data_val = load_dataset('./dataset/newDatasetSplit/val')

# --- MAP EACH INPUT TO ITS TEMPLATE IMAGE ---
def map_to_target(img, label):
    target_img = tf.convert_to_tensor([template_images[int(l)] for l in label.numpy()], dtype=tf.float32)
    return img, target_img

def tf_wrapper(img, label):
    img, target_img = tf.py_function(map_to_target, inp=[img, label], Tout=(tf.float32, tf.float32))
    img.set_shape((None, img_height, img_width, 3))
    target_img.set_shape((None, img_height, img_width, 3))
    return img, target_img

paired_train = data_train.map(tf_wrapper).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
paired_val = data_val.map(tf_wrapper).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# --- MODEL: ENCODER-DECODER ---
def build_image_to_image_model():
    inputs = Input(shape=(img_height, img_width, 3))
    x = layers.Rescaling(1.0)(inputs)  # Images already normalized

    # Encoder
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
    skip1 = x
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    skip2 = x
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    skip3 = x
    x = layers.MaxPooling2D()(x)

    # Decoder
    x = layers.Conv2DTranspose(128, 3, strides=2, activation='relu', padding='same')(x)
    x = layers.Concatenate()([x, skip3])
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)

    x = layers.Conv2DTranspose(64, 3, strides=2, activation='relu', padding='same')(x)
    x = layers.Concatenate()([x, skip2])
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)

    x = layers.Conv2DTranspose(32, 3, strides=2, activation='relu', padding='same')(x)
    x = layers.Concatenate()([x, skip1])
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)

    outputs = layers.Conv2D(3, 3, activation='sigmoid', padding='same')(x)

    return models.Model(inputs, outputs)

# --- COMPILE ---
model = build_image_to_image_model()
model.compile(optimizer='adam', loss='mse')

# --- TRAIN ---
EPOCHS = 32
history = model.fit(
    paired_train,
    validation_data=paired_val,
    epochs=EPOCHS
)

# --- PLOT RESULTS ---
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()

plt.subplot(1, 2, 2)
for image, label in data_val.take(1):
    pred = model.predict(image / 255.0)
    for i in range(min(4, len(image))):
        plt.subplot(2, 4, i + 1)
        plt.imshow(image[i].numpy().astype("uint8"))
        plt.axis('off')
        plt.title("Input")
        plt.subplot(2, 4, i + 5)
        plt.imshow(pred[i])
        plt.axis('off')
        plt.title("Output")

plt.tight_layout()
plt.show()

# --- SAVE MODEL ---
model.save("image_to_image_model.keras")
print("Model saved!")


2025-05-01 13:44:25.477933: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746107069.401809    9152 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746107070.490967    9152 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746107079.162532    9152 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746107079.162589    9152 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746107079.162595    9152 computation_placer.cc:177] computation placer alr

Found 17125 files belonging to 25 classes.


I0000 00:00:1746107243.574640    9152 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Found 3675 files belonging to 25 classes.
Epoch 1/32


I0000 00:00:1746107269.772319    9661 service.cc:152] XLA service 0x7fead00064a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746107269.772489    9661 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-05-01 13:47:50.888728: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1746107272.526819    9661 cuda_dnn.cc:529] Loaded cuDNN version 90501
2025-05-01 13:48:00.253026: E external/local_xla/xla/service/slow_operation_alarm.cc:73] Trying algorithm eng20{k2=2,k4=1,k5=0,k6=0,k7=0,k19=0} for conv %cudnn-conv-bias-activation.21 = (f32[8,32,128,128]{3,2,1,0}, u8[0]{0}) custom-call(f32[8,3,128,128]{3,2,1,0} %bitcast.4115, f32[32,3,3,3]{3,2,1,0} %bitcast.4106, f32[32]{0} %bitcast.4531), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="

1353/2141 ━━━━━━━━━━━━━━━━━━━━ 4:07 314ms/step - loss: 0.3107

: 